run this to see the webui:
docker build -t chatbot-ollama . # only for first time building in repo
docker run -p 3000:3000 chatbot-ollama



In [ ]:
import ollama
import builtins
import contextlib
import io
from typing import Any
import re
import textwrap  # Ensure this is added to your imports
import traceback  # Ensure this is imported at the top

# NEW: Import MCP server tools (assuming mcp is installed)
from fastmcp import FastMCP

mcp = FastMCP("Echo")

# NEW: Attempt to import nba_api; if not available, set to None.
try:
    from nba_api.stats.endpoints import commonplayerinfo
except ImportError:
    commonplayerinfo = None

# NEW: Attempt to import CodeAct module; if not available, set to None.
try:
    import codeact
except ImportError:
    codeact = None
from nba_api.stats.endpoints import LeagueDashTeamStats
import pandas as pd
import numpy as np
from langchain.chat_models import init_chat_model
from langgraph_codeact import create_codeact

def get_nba_endpoint_metadata_manual(season: str) -> dict:
    """
    Fetch metadata from the LeagueDashTeamStats endpoint for the given season.
    
    Parameters:
        season (str): Season string in the format "YYYY-YY" (e.g., "2021-22").
        
    Returns:
        dict: Contains:
            - "columns": List of column names in the returned DataFrame.
            - "shape": Tuple (number of rows, number of columns).
            - "head": The first 3 rows (as a list of dicts).
            - "dtypes": A dictionary mapping each column to its data type.
    """
    # Validate season format; expect format like "2021-22"
    if not isinstance(season, str) or len(season) != 7 or '-' not in season:
        raise ValueError("Invalid season format. Expected format 'YYYY-YY'.")
    
    # Use the correct parameter name 'per_mode' instead of 'per_game'
    stats = LeagueDashTeamStats(
        per_mode='TOTAL',         # Use 'TOTAL' to fetch total points
        target_level='SEASON',    # Ensure data is aggregated per season
        season=season
    )
    # nba_api returns data as a list of DataFrames; we take the first one.
    df = stats.get_data_frames()[0]
    
    metadata = {
        "columns": df.columns.tolist(),
        "shape": df.shape,
        "head": df.head(3).to_dict('records'),
        "dtypes": df.dtypes.apply(lambda dt: str(dt)).to_dict()
    }
    return metadata




@mcp.tool(
    name="get_nba_player_stats",
    description="Fetch NBA player statistics for a given player ID using the NBA API. Returns key player details in a formatted string."
)
async def get_nba_player_stats(player_id: int) -> str:
    """
    Uses nba_api to retrieve and format NBA player statistics.
    If the nba_api package is not installed, returns an error message.
    """
    if commonplayerinfo is None:
        return "Error: nba_api package is not installed. Please install it to use this feature."
    try:
        # Create a CommonPlayerInfo instance for the given player_id.
        player_info = commonplayerinfo.CommonPlayerInfo(player_id=player_id)
        # Retrieve normalized dictionary containing player info.
        info_dict = player_info.get_normalized_dict()
        # Format and return the info.
        return f"NBA Player Stats for ID {player_id}:\n{info_dict}"
    except Exception as e:
        return f"Error fetching NBA player stats: {e}"

@mcp.tool(
    name="generate_mcp_query",
    description="Generates a Python query for the MCP protocol using CodeAct from a natural language description."
)
async def generate_mcp_query(query: str) -> str:
    """
    Uses CodeAct (a hypothetical module) to generate a Python query for MCP.
    If CodeAct is not installed, returns an error message.
    """
    if codeact is None:
        return "Error: CodeAct module is not available. Please install or configure it."
    try:
        # Generate a Python query using CodeAct.
        response = codeact.chat(model="codeact-model:latest", messages=[
            {'role': 'user', 'content': f'Generate an MCP Python query for: {query}'}
        ])
        generated_query = response['message']['content']
        return f"Generated MCP Python Query:\n{generated_query}"
    except Exception as e:
        return f"Error generating MCP query: {e}"

@mcp.tool(
    name="generate_and_run_nba_query",
    description="Generates and runs Python code to query NBA data using CodeAct. It returns the generated code and its execution output."
)
async def generate_and_run_nba_query(natural_language_query: str) -> str:
    """
    Steps:
    1. Use CodeAct to generate Python code that queries NBA API based on the user's natural language query.
    2. Extract generated Python code from the response.
    3. Execute the generated code safely within a sandbox.
    4. Return the generated Python code and its execution output.
    """
    # Check for CodeAct availability
    if codeact is None:
        return "Error: CodeAct module not available."

    # Step 1: Generate Python code via CodeAct using the NBA API
    try:
        codeact_response = codeact.chat(
            model="codeact-model:latest",
            messages=[
                {'role': 'user', 'content': f'Generate Python code using nba_api to fetch NBA player stats for the query: "{natural_language_query}"'}
            ]
        )
        raw_generated_code = codeact_response['message']['content']
    except Exception as e:
        return f"CodeAct generation error: {e}"

    # Step 2: Extract Python code block from response
    try:
        generated_code = extract_python_code(raw_generated_code)
    except ValueError as e:
        return f"Error extracting code: {e}"

    # Step 3: Verify the generated code compiles
    try:
        compile(generated_code, '<string>', 'exec')
    except Exception as e:
        return f"Compilation error: {e}"

    # Step 4: Execute the generated code in the sandbox.
    # We pass in the nba_api object as part of the execution environment.
    execution_output, _ = sandbox_run(generated_code, {"commonplayerinfo": commonplayerinfo})
    
    # Return both the generated code and its output.
    return (
        f"Generated Python Code:\n```python\n{generated_code}\n```\n\n"
        f"Execution Output:\n{execution_output}"
    )

@mcp.tool(
    name="get_nba_endpoint_metadata",
    description="Retrieves metadata from the LeagueDashTeamStats endpoint for a given season. Returns column names, shape, sample rows, and data types."
)
async def get_nba_endpoint_metadata(season: str) -> str:
    """
    Uses the helper function to fetch metadata about the LeagueDashTeamStats endpoint for the given season.
    
    Parameters:
        season (str): Season string (e.g., "2021-22").
        
    Returns:
        str: A formatted string (JSON) with the metadata.
    """
    try:
        metadata = get_nba_endpoint_metadata_manual(season)
        import json
        formatted = json.dumps(metadata, indent=4)
        return f"Metadata for LeagueDashTeamStats for season {season}:\n{formatted}"
    except Exception as e:
        return f"Error retrieving metadata: {e}\n{traceback.format_exc()}"



def extract_python_code(response_text: str) -> str:
    """
    Extracts and returns the Python code contained in a markdown code block from the response text.
    Searches for a block delimited by '```python' and '```'. 
    Raises a ValueError if no valid Python code block is found.
    """
    match = re.search(r"```python\n(.*?)\n```", response_text, re.DOTALL)
    if match:
        # Return only the code inside the markdown code block, stripped of extra whitespace.
        return match.group(1).strip()
    else:
        raise ValueError("No valid Python code block found in the response.")

# ---------------------------
# Function: generate_python_function
# ---------------------------
def generate_python_function(description): 
    response = ollama.chat(model='deepseek-r1:latest', messages=[
        {'role': 'user', 'content': f'Generate a Python function for: {description}'}
    ])   
    return response['message']['content']

# ---------------------------
# Function: sandbox_run
# ---------------------------
def sandbox_run(code: str, env: dict[str, Any]) -> tuple[str, dict[str, Any]]:
    """
    Execute code safely and capture stdout.
    """
    _locals = env.copy()
    original_keys = set(_locals.keys())
    try:
        f = io.StringIO()
        with contextlib.redirect_stdout(f):
            exec(code, {"__builtins__": builtins.__dict__}, _locals)
        result_output = f.getvalue().strip() or "<no output>"
    except Exception as e:
        # Capture and include the full traceback for debugging purposes
        result_output = f"Error during execution: {e}\n{traceback.format_exc()}"
    new_vars = {k: _locals[k] for k in _locals.keys() - original_keys}
    return result_output, new_vars


# ---------------------------
# Main execution to test integration
# ---------------------------
def main():
    """
    Main execution function.
    If '--mcp' is passed as a command-line argument, the MCP server starts.
    Otherwise, it runs the generate–extract–run test flow using an NBA query for average team points.
    Additionally, it calls the manual metadata function for debugging.
    
    The expected generated function is 'average_team_points' which takes a season string (e.g., "2021-22")
    and returns the average team points.
    """
    import sys
    if "--mcp" in sys.argv:
        print("Starting MCP server (using stdio transport)...")
        mcp.run(transport='stdio')
        return

    # --- Manual Test of the Metadata Helper ---
    try:
        meta = get_nba_endpoint_metadata_manual("2021-22")
        import json
        print("Manual Endpoint Metadata:")
        print(json.dumps(meta, indent=4))
    except Exception as e:
        print("Error retrieving manual metadata:", e)
    
    # --- Existing test execution for generated code ---
    description = (
        "Generate a Python function using nba_api that calculates the average NBA team points "
        "total per season for a given season (e.g., '2021-22'). The function should be named "
        "'average_team_points' and accept a season string as input, then return the average points "
        "from the 'PTS' column using the LeagueDashTeamStats endpoint from nba_api.stats.endpoints."
    )
    
    raw_generated_response = generate_python_function(description)
    print("Raw Generated Response:\n", raw_generated_response)
    
    try:
        generated_code = extract_python_code(raw_generated_response)
    except ValueError as e:
        print("Error extracting code:", e)
        return

    print("\n[DEBUG] Extracted Code Representation:")
    print(repr(generated_code))
    
    try:
        compile(generated_code, '<string>', 'exec')
    except Exception as e:
        print("Compilation error in generated code:", e)
        return

    # Test code calls the generated function with a sample season.
    test_code = f"""
{generated_code}

# Example usage of the generated NBA function
if __name__ == '__main__':
    # Call the function with a sample season string (e.g., "2021-22")
    avg_points = average_team_points("2021-22")
    print('Average NBA Team Points in 2021-22:', avg_points)
"""
    print("\n[DEBUG] Complete Test Code:")
    print(repr(test_code))

    output, new_vars = sandbox_run(test_code, {})
    print("\nSandbox Execution Output:\n", output)

if __name__ == "__main__":
    try:
        meta = get_nba_endpoint_metadata_manual("2021-22")
        import json
        print("Average NBA Team Points Endpoint Metadata (Manual Test):")
        print(json.dumps(meta, indent=4))
    except Exception as e:
        print("Error during manual metadata test:", e)
    main()




Error during manual metadata test: LeagueDashTeamStats.__init__() got an unexpected keyword argument 'per_mode'
Error retrieving manual metadata: LeagueDashTeamStats.__init__() got an unexpected keyword argument 'per_mode'


[03/30/25 09:48:38] INFO     HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"   ]8;id=505096;file://c:\Users\ghadf\anaconda3\envs\data_science_ollama\lib\site-packages\httpx\_client.py\_client.py]8;;\:]8;id=407474;file://c:\Users\ghadf\anaconda3\envs\data_science_ollama\lib\site-packages\httpx\_client.py#1025\1025]8;;\

Raw Generated Response:
 <think>
Okay, I need to create a Python function called average_team_points that calculates the average NBA team points total per season for a given year, like '2021-22'. The user wants this function to use the nba_api library and specifically the LeagueDashTeamStats endpoint.

First, I should import the necessary modules. Since we're using nba_api, I'll need to import the client from nba_api, probably just as Client. Also, from the statistics endpoint, I'll need client as well because we'll be working with stats clients in our code.

Next, the function will take a single parameter: season. That's straightforward. The function should return the average points per team for that season.

I remember that NBA data is structured into different endpoints and seasons have specific codes. For example, '2021-22' corresponds to the 2021-22 season in the nba_api system, which I think has a code of 345. So, when making API calls, we need to specify this code.

Now, using t